In [1]:
import torch
import torchvision
import cv2
import numpy as np
import pandas as pd
import os
import time
import random
import copy
from Data_PreProcess import Data_preprocess as dp
import json
from data_loader import DataGenerate,collate
from torch.utils.data import Dataset, DataLoader
from Generate_Anchor import Generate_Anchor
from torchvision import models
from Net3 import Net
from Eval import Eval as eval_model

In [2]:
batch_size = 1
input_size = 864
anchor_info = {
    "anchor_size_3" : [[350, 350],[425, 425],[475, 475],[400, 300],[300, 400],[450, 350],[350, 450]],
    "anchor_size_1" : [[ 20,  20],[ 30,  30],[ 45,  45],[ 70,  70],[100, 100],[100,  60],[ 60, 100]],
    "anchor_size_2" : [[150, 150],[225, 225],[300, 300],[225, 150],[150, 225],[100, 200],[200, 100]],
    "feat_size_1":(input_size//8,input_size//8),
    "feat_size_2":(input_size//16,input_size//16),
    "feat_size_3":(input_size//32,input_size//32)
}
model = Net(input_size=864,batch_size=1, anchor_info=anchor_info)
model.load_state_dict(torch.load(r"E:\cancer\model\model_include3\eps=3.t7"),strict = False)
model = model.cuda()
model.eval()
pass

In [3]:
gt = pd.read_csv(r"E:\cancer\imglabel\val\3_gt.csv", index_col=0)
#gt = pd.read_csv(r"E:\cancer\imglabel\groundTruth\5_gt.csv", index_col=0)
gt = gt.reindex(columns=["x1","y1","x2","y2"])
val_generate = DataLoader(DataGenerate(batch_size=batch_size, input_size=input_size,is_training=False,gt = gt
                                        ), batch_size = batch_size, drop_last = True,collate_fn=collate)


In [4]:
def collate(batch):
    imgbatch = np.stack([b[0] for b in batch], 0)
    imgbatch = torch.from_numpy(imgbatch).cuda().float()
    imgbatch = imgbatch.permute((0, 3, 1, 2))
    Labels = batch[0][1]
    return imgbatch, Labels

with torch.no_grad():
    t1 = time.time()
    for idx,data in enumerate(val_generate):
        #img = data[0][0].cpu().numpy().transpose(1,2,0).astype(np.uint8)[np.newaxis,]
        #img = torch.from_numpy(img).cuda().float()
        t2 = time.time()
        s1_res, s2_res = model(data[0],threshold=0.75)
        s1_res = s1_res.detach().cpu().numpy()
        s1_res = s1_res[s1_res[:,-1] > 0.65]
        s1_res = s1_res[dp.py_cpu_nms(s1_res)]
        s2_res = s2_res.detach().cpu().numpy()
        #s2_res = s2_res[s2_res[:,-1] > 0.5]
        s2_res = s2_res[dp.py_cpu_nms(s2_res)]
        print("t1",time.time() - t1)
        print("t2",time.time() - t2)
        img = data[0][0].cpu().numpy().transpose(1,2,0).astype(np.uint8)
        img = dp.draw_bboxs(img, data[1][0].reshape(-1,5).astype(np.int),color=(255,0,255))
        t1 = time.time()
        img = dp.draw_bboxs(img, s1_res,color=(0,0,255))
        img = dp.draw_bboxs(img, s2_res,color=(0,255,0))
        print("s1_res\n ",s1_res)
        print("s2_res\n ",s2_res)
        dp.imgDisplay(img)
        if(idx>20):break

t1 1.3289484977722168
t2 1.0080962181091309
s1_res
  [[3.6849598e+01 7.8270453e+02 9.8034203e+01 8.3723175e+02 7.7062941e-01]
 [7.9383002e+02 5.7951520e+02 8.4193732e+02 6.1638531e+02 7.5933671e-01]
 [2.9884056e+01 8.1791327e+02 1.0994558e+02 8.5231329e+02 7.5452566e-01]]
s2_res
  [[2.6258595e+01 8.1043695e+02 8.8031769e+01 8.5719885e+02 4.8122784e-01]]
t1 6.549618721008301
t2 0.08121633529663086
s1_res
  [[8.0092450e+02 6.6749371e+02 8.3718561e+02 7.0954681e+02 8.0820441e-01]
 [4.1191382e+02 1.2069160e+02 4.8873663e+02 1.8738643e+02 7.7257794e-01]
 [6.1704462e+02 7.9839893e+02 6.5347089e+02 8.2950934e+02 7.7018243e-01]
 [7.7003687e+02 6.7466669e+02 8.2989813e+02 7.2952808e+02 7.5028503e-01]]
s2_res
  [[408.10263    103.13423    484.44138    181.51866      0.53681827]]
t1 1.1368489265441895
t2 0.08272051811218262
s1_res
  [[7.7653278e+02 4.4601562e+01 8.2985870e+02 8.5206596e+01 9.2130697e-01]
 [7.7643665e+02 6.9265198e+01 8.2773608e+02 1.0865672e+02 8.8805568e-01]
 [8.1491040e+02 7.41

 [7.6844135e+02 1.6312082e+02 8.2162152e+02 2.0292059e+02 7.6386124e-01]]
s2_res
  []
t1 0.7717421054840088
t2 0.07770967483520508
s1_res
  [[7.9427710e+02 6.9817670e+02 8.4478876e+02 7.4199194e+02 8.3364618e-01]
 [7.8912891e+02 6.1592841e+02 8.3966931e+02 6.6367499e+02 8.0851507e-01]
 [7.9536658e+02 7.2506830e+02 8.4432385e+02 7.6926727e+02 7.9489076e-01]
 [8.3165527e+02 6.4005389e+02 8.5973590e+02 6.6740857e+02 7.7108008e-01]]
s2_res
  []
t1 0.9219784736633301
t2 0.07921099662780762
s1_res
  [[7.86910583e+02 1.15309944e+02 8.36692383e+02 1.56239349e+02
  8.43833089e-01]
 [7.94240051e+02 1.41266205e+02 8.39063171e+02 1.76283676e+02
  8.24161708e-01]
 [2.38224182e+02 7.96086426e+02 3.01729645e+02 8.34519958e+02
  8.21677208e-01]
 [2.67033539e+02 7.86542786e+02 3.04190125e+02 8.18749084e+02
  7.88420320e-01]
 [2.96605438e+02 5.74821289e+02 3.54190948e+02 6.28927124e+02
  7.63534129e-01]
 [7.94364258e+02 1.65346695e+02 8.40258972e+02 1.96847260e+02
  7.61527121e-01]
 [7.91623535e+02 4.73